# Preprocesamiento con Pandas y Sklearn
**Autor:** José A. Troyano    &nbsp;&nbsp;&nbsp; **Última modificación:** 12/12/2023

------------------------------------------------
## Contenido
1. <a href="#normalizacion">Normalización <br>
    1.1. <a href="#min_max_z_score">Min-max y _Z-score_  </a> <br>
    1.2. <a href="#norm_sklearn">Normalización con _sklearn_ </a> <br>
2. <a href="#agrupacion">Agrupación </a> <br>
    2.1. <a href="#groupby">Cálculo de grupos con _groupby_ </a> <br>
    2.2. <a href="#agregacion">Agregación a partir de grupos </a> <br>
    2.3. <a href="#varias_claves">Agrupación por más de una clave </a> <br>
3. <a href="#codificacion"> Codificación de atributos discretos </a> <br>
    3.1. <a href="#tipos_atributos">Tipos de atributos </a> <br>
    3.2. <a href="#label_encoding">Codificación _label encoding_</a> <br>
    3.3. <a href="#one_hot_encoding">Codificación _one hot encoding_</a> <br>
    3.4. <a href="#reduccion_one_hot">Reducción de dimensionalidad de la codificación _one hot_ </a> <br>
4. <a href="#valores_ausentes"> Cálculo de valores ausentes </a> <br>
    4.1. <a href="#global">Rellenar con  valores globales </a> <br>
    4.2. <a href="#por_grupo">Rellenar con valores específicos para grupos </a> <br>
-------------------------------------------------

Trabajaremos con un dataset pequeño muy usado para introducir los conceptos básicos de minería de datos: el dataset <code>weather.csv</code>. Tiene solo 5 atributos y 14 instancias:

|outlook|temperature|humidity|windy|play|
|-------|-----------|--------|-----|----|
|sunny|85|85|FALSE|no|
|sunny|80|90|TRUE|no|
|overcast|83|86|FALSE|yes|
|rainy|70|96|FALSE|yes|
|rainy|68|80|FALSE|yes|
|rainy|65|70|TRUE|no|
|overcast|64|65|TRUE|yes|
|sunny|72|95|FALSE|no|
|sunny|69|70|FALSE|yes|
|rainy|75|80|FALSE|yes|
|sunny|75|70|TRUE|yes|
|overcast|72|90|TRUE|yes|
|overcast|81|75|FALSE|yes|
|rainy|71|91|TRUE|no|

Además de la versión original, dispondremos de una versión ofuscada en la que se han eliminado ciertos valores (<code>weather_missing.csv</code>) para probar algunas técnicas simples de cálulo de valores ausentes.

Empezaremos por la importación de Pandas y Numpy:

In [49]:
import pandas as pd
import numpy as np

## 1. Normalización <a name="normalizacion"> </a>

In [50]:
# EJERCICIO: cargar los datos de 'weather.csv' en el dataframe 'DATOS'
DATOS = pd.read_csv('./weather.csv')

### 1.1. Min-max y _Z-score_ <a name="min_max_z_score"> </a>
Algunos algoritmos de aprendizaje necesitan que los distintos atributos estén en un rango de valores similar (<code>SVM</code> es uno de ellos. Probaremos dos técnicas simples de normalización de datos: 
- Normalización _min-max_: restar el mínimo y dividir por la diferencia entre máximo y mínimo (produce valores entre 0 y 1)
- Normalización _Z-score_: restar la media y dividir por la desviación estándar (puede producir valores negativos)


In [51]:
# EJERCICIO: normalizar los valores de 'humidity' entre [0,1] con la técnica _min-max_ y guardarlos en la columna 'humidity_mm'

def min_max(df, col, col2):
    df[col2] = (df[col] - min(df[col]))/(max(df[col])-min(df[col]))
    return df

min_max(DATOS, 'humidity', 'humidity_mm')

,outlook,temperature,humidity,windy,play,humidity_mm
0,sunny,85,85,False,no,0.645161
1,sunny,80,90,True,no,0.806452
2,overcast,83,86,False,yes,0.677419
3,rainy,70,96,False,yes,1.000000
4,rainy,68,80,False,yes,0.483871
5,rainy,65,70,True,no,0.161290
6,overcast,64,65,True,yes,0.000000
7,sunny,72,95,False,no,0.967742
8,sunny,69,70,False,yes,0.161290
9,rainy,75,80,False,yes,0.483871


In [52]:
# EJERCICIO: 
#    - Normalizar los valores de 'humidity' con la técnica z-score y guardarlos en la columna 'humidity_zs'
#      restar la media y dividir por la desviación estándar (puede producir valores negativos)
#    - Comparar la media y la desviación estándar de ambas columnas

def z_score(df, col, col2):
    df[col2] = (df[col] - np.average(df[col]))/(np.std(df[col]))
    return df

z_score(DATOS, 'humidity', 'humidity_zs')
    

,outlook,temperature,humidity,windy,play,humidity_mm,humidity_zs
0,sunny,85,85,False,no,0.645161,0.338726
1,sunny,80,90,True,no,0.806452,0.843212
2,overcast,83,86,False,yes,0.677419,0.439623
3,rainy,70,96,False,yes,1.000000,1.448595
4,rainy,68,80,False,yes,0.483871,-0.165760
5,rainy,65,70,True,no,0.161290,-1.174731
6,overcast,64,65,True,yes,0.000000,-1.679217
7,sunny,72,95,False,no,0.967742,1.347697
8,sunny,69,70,False,yes,0.161290,-1.174731
9,rainy,75,80,False,yes,0.483871,-0.165760


### 1.2. Normalización con <code>sklearn</code> <a name="norm_sklearn"> </a>

Sklearn proporciona algunos métodos de normalización, probaremos también cómo usar algunos de ellos:
- <code>MaxAbsScaler</code>: desplaza y escala para que el máximo valor absoluto sea 1. Los valores resultantes quedan, por tanto, dentro del rango [-1,1]
- <code>MinMaxScaler</code>: resta el mínimo y divide por la amplitud (máximo-mínimo)
- <code>StandardScaler</code>: resta la media y divide por la desviación típica (es el _Z-score_)

Será la primera vez que usemos un estimador de Sklear. Antes de empezar tendremos que buscar la respuesta a estas dos preguntas:

- ¿Qué es <code>fit</code>?

    El método <code>fit</code> se utiliza para ajustar un estimador (modelo) a los datos de entrenamiento. 


- ¿Qué es <code>transform</code>?

    El método <code>transform</code> se utiliza para aplicar la transformación aprendida por el estimador (modelo) a los datos. Es decir, este método se utiliza después de haber ajustado el modelo a los datos de entrenamiento utilizando <code>fit</code>.


- ¿Qué es <code>fit_transform</code>?

    El método <code>fit_transform</code>, que combina los pasos de ajuste <code>fit</code> y transformación <code>transform</code> en uno solo.
    

- ¿Qué es <code>predict</code>?

    El método <code>predict</code> realiza la predicción final sobre los datos de test

In [53]:
# EJERCICIO: importar desde sklearn los elementos que necesitemos
from sklearn.preprocessing import MaxAbsScaler, MinMaxScaler, StandardScaler


In [54]:
# EJERCICIO: aplicar las transformacines necesarias para crear las siguientes columnas

#    - temp_maxabs: MaxAbsScaler sobre la columna 'temperature'

DATOS['temp_maxabs'] = MaxAbsScaler().fit_transform(DATOS[['temperature']])

#    - temp_minmax: MinMaxScaler sobre la columna 'temperature'

DATOS['temp_minmax'] = MinMaxScaler().fit_transform(DATOS[['temperature']])


#    - temp_sca: StandardScaler sobre la columna 'temperature'

DATOS['temp_sca'] = StandardScaler().fit_transform(DATOS[['temperature']])

DATOS

,outlook,temperature,humidity,windy,play,humidity_mm,humidity_zs,temp_maxabs,temp_minmax,temp_sca
0,sunny,85,85,False,no,0.645161,0.338726,1.000000,1.000000,1.804715
1,sunny,80,90,True,no,0.806452,0.843212,0.941176,0.761905,1.015152
2,overcast,83,86,False,yes,0.677419,0.439623,0.976471,0.904762,1.488890
3,rainy,70,96,False,yes,1.000000,1.448595,0.823529,0.285714,-0.563974
4,rainy,68,80,False,yes,0.483871,-0.165760,0.800000,0.190476,-0.879799
5,rainy,65,70,True,no,0.161290,-1.174731,0.764706,0.047619,-1.353537
6,overcast,64,65,True,yes,0.000000,-1.679217,0.752941,0.000000,-1.511449
7,sunny,72,95,False,no,0.967742,1.347697,0.847059,0.380952,-0.248148
8,sunny,69,70,False,yes,0.161290,-1.174731,0.811765,0.238095,-0.721886
9,rainy,75,80,False,yes,0.483871,-0.165760,0.882353,0.523810,0.225589


## 2. Agrupación <a name="agrupacion"> </a>

Pandas permite agrupar los dataframes en función de muchos criterios. Nosotros solo veremos uno: en función de los valores de una o más columnas. A partir de la agrupación resultante, se pueden calcular valores asociados a estos grupos como, por ejemplo:
- Media de un atributo continuo en función de una agrupación según otros atributos
- Valor más repetido de un atributo discreto en función de una agrupación según otros atributos

Como veremos más adelante, las informaciones anteriores pueden ser de utilidad a la hora de rellenar un valor ausente en función de los valores de otros atributos. 

In [55]:
# En algunos ejercicios usaremos la función 'moda' que calcula el valor más repetido de una serie. Aunque
# está implementada en algunos paquetes (como statistics o scipy), usaremos esta implementación simple que asegura
# que siempre se obtiene un único valor.
def moda(serie): 
    return max(set(serie), key=list(serie).count)

### 2.1. Cálculo de grupos con <code>groupby</code> <a name="groupby"> </a>

In [56]:
# EJERCICIO: agrupar los registros del data frame por el atributo 'outlook' y guardar en 'grupos_outlook' 

grupos_outlook = DATOS.groupby('outlook')
grupos_outlook

In [57]:
# TEST: de la agrupación por 'outlook' 
print(grupos_outlook)
for k,g in grupos_outlook: #<-- El groupby es un iterador que construye pares clave, valor (k,g)
    print(k)
    print (g[['humidity', 'windy']])

overcast
    humidity  windy
2         86  False
6         65   True
11        90   True
12        75  False
rainy
    humidity  windy
3         96  False
4         80  False
5         70   True
9         80  False
13        91   True
sunny
    humidity  windy
0         85  False
1         90   True
7         95  False
8         70  False
10        70   True


### 2.2. Agregación a partir de grupos <a name="agregacion"> </a>

In [58]:
# EJERCICIO: calcular la media del atributo continuo 'humidity' para todos los grupos de 'grupos_outlook' usando np.mean

grupos_outlook[['humidity']].agg(np.mean)

#DATOS.groupby('outlook')[['humidity']].agg(np.mean)

#con dos atributos --> grupos_outlook[['humidity', 'temperature']].agg(np.mean)

,humidity
outlook,
overcast,79.0
rainy,83.4
sunny,82.0


In [59]:
# EJERCICIO: calcular la media del atributo continuo 'humidity' para todos los grupos de 'grupos_outlook' usando el métodod mean de las series

grupos_outlook['humidity'].agg('mean')


outlook
overcast    79.0
rainy       83.4
sunny       82.0
Name: humidity, dtype: float64

In [60]:
# EJERCICIO: calcular el valor más frecuente del atributo discreto 'windy' para todos los grupos de 'gupos_outlook'

grupos_outlook['windy'].agg(moda)


outlook
overcast    False
rainy       False
sunny       False
Name: windy, dtype: bool

In [61]:
# EJERCICIO: calcular, al mismo tiempo, la media de 'humidity' y la moda de 'windy' 

DATOS.groupby(['outlook']).agg({'humidity': np.mean,
                               'windy': moda})


,humidity,windy
outlook,,
overcast,79.0,False
rainy,83.4,False
sunny,82.0,False


### 2.3. Agrupación por más de una clave <a name="varias_claves"> </a>

In [62]:
# EJERCICIO:
#   - Agrupar por 'outlook' y 'windy', ...
#   - ... agregando 'humidity' con la media, y 'play' con la moda 

grupos_out_wind = DATOS.groupby(['outlook', 'windy'])
grupos_out_wind.agg({'humidity': np.mean,'windy': moda})

humidity  windy
outlook  windy                  
overcast False  80.500000  False
         True   77.500000   True
rainy    False  85.333333  False
         True   80.500000   True
sunny    False  83.333333  False
         True   80.000000   True

In [63]:
# EJERCICIO: repetir el ejercicio anterior aplanando el índice jerárquico


## 3. Codificación de atributos discretos <a name="codificacion"> </a>

En esta sección veremos cómo codificar atributos discretos mediante valores numéricos. Esto es necesario porque muchos de los algoritmos de aprendizaje solo soportan atributos numéricos. En concreto, los algoritmos implemenatdos en Sklearn solo pueden manejar atributos numéricos.

- ¿Qué es <code>drop()</code>?

    Método para eliminar columnas.
    
    
- ¿Qué es <code>dropna()</code>?

    Método para eliminar filas incompletas.
    
   
- ¿Qué es el parámetro <code>inplace</code>?

    El parámetro inplace generalmente está a False, no modifica el df real. Sin embargo, si lo ponemos a True si realiza la modificación sobre el df.
    

In [64]:
# Cargamos los datos de 'food-preference.csv' en el dataframe 'DATOS'
DATOS = pd.read_csv('./food-preference.csv')
DATOS.drop(columns=['Timestamp', 'Participant_ID'], inplace=True)
DATOS.dropna(inplace=True)
DATOS[:5]

,Gender,Nationality,Age,Food,Juice,Dessert
0,Male,Indian,24,Traditional food,Fresh Juice,Maybe
1,Female,Indian,22,Western Food,Carbonated drinks,Yes
2,Male,Indian,31,Western Food,Fresh Juice,Maybe
3,Female,Indian,25,Traditional food,Fresh Juice,Maybe
4,Male,Indian,27,Traditional food,Fresh Juice,Maybe


### 3.1. Tipos de atributos <a name="tipos_atributos"> </a>

In [65]:
# EJERCICIO: crear las matrices 'DATOS_discretos' y 'DATOS_numericos' con los atributos discretos y numéricos, respectivamente, de 'DATOS'

DATOS_discretos = DATOS.select_dtypes(include='object')#.copy()

print(DATOS_discretos.info())

DATOS_numericos = DATOS.select_dtypes(include=[np.number])#.copy()

print(DATOS_numericos.info())

<class 'pandas.core.frame.DataFrame'>
Index: 284 entries, 0 to 287
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Gender       284 non-null    object
 1   Nationality  284 non-null    object
 2   Food         284 non-null    object
 3   Juice        284 non-null    object
 4   Dessert      284 non-null    object
dtypes: object(5)
memory usage: 13.3+ KB
None
<class 'pandas.core.frame.DataFrame'>
Index: 284 entries, 0 to 287
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Age     284 non-null    int64
dtypes: int64(1)
memory usage: 4.4 KB
None


In [66]:
#Agregación
DATOS_discretos['Nationality'].value_counts()

Nationality
Indian         238
Malaysian       10
Indonesia        7
Pakistani        3
MY               2
Japan            2
Maldivian        2
Pakistani        2
Pakistan         1
Korean           1
Mauritian        1
China            1
Malaysia         1
Indonesian       1
Indonesain       1
Seychellois      1
Nigerian         1
Algerian         1
Maldivian        1
Canadian         1
Malaysia         1
MALAYSIAN        1
Indonesian       1
Malaysian        1
Tanzanian        1
Yemen            1
Name: count, dtype: int64

In [67]:
# EJERCICIO: mostrar el número de 'levels' para cada atributo discreto
# levels son los distintos valores que puede adquerir el atributo

levels = pd.DataFrame(columns=['Numero', 'Levels'])

for atributo in DATOS_discretos.columns.values:
    valores = list(DATOS_discretos[atributo].value_counts().keys())
    levels.loc[atributo] = (len(valores),valores)
    
levels

,Numero,Levels
Gender,2,"[Female, Male]"
Nationality,26,"[Indian, Malaysian, Indonesia, Pakistani, MY, ..."
Food,2,"[Traditional food, Western Food]"
Juice,2,"[Fresh Juice, Carbonated drinks]"
Dessert,3,"[Maybe, Yes, No]"


En general, nos podemos encontrar con tres tipos de atributos discretos:
- **Binarios (o categóricos con dos _levels_)** : como el atributo <code>play</code> del ejemplo
- **Categóricos**: como el atributo <code>outlook</code> del ejemplo
- **Ordinales**: sería un atributo categórico en el que, además, hay una relación de orden. Por ejemplo sería un atributo <code>nivel_humedad</code> con valores <code>['bajo', 'medio', 'alto']</code>.

Los atributos ordinales se pueden codificar mediante un único atributo numérico, ya que la relación de orden se mantiene en la representación numérica. A este tipo de codificación se le denomina _label encoding_.

Los categóricos, sin embargo, no se pueden codificar con un número, ya que el algoritmo de aprendizaje asumiría una relación de orden que no existe. En este caso se debe utilizar una codificación en varias columnas, el denominado _one-hot encoding_.

Los binarios son realmente categóricos, pero podemos intentar codificarlos con un único atributo numérico que tome los valores $0$ y $1$.

### 3.2. Codificación _label encoding_<a name="label_encoding"> </a>

In [68]:
# EJERCICIO: codificar mediante label encoding los atributos binarios

DATOS['Gender'] = DATOS['Gender'].map({'Female':0, 'Male':1})
DATOS['Food'] = DATOS['Food'].map({'Traditional food':0, 'Western Food':1})
DATOS['Juice'] = DATOS['Juice'].map({'Fresh Juice':0, 'Carbonated drinks':1 })
DATOS[:10]



,Gender,Nationality,Age,Food,Juice,Dessert
0,1,Indian,24,0,0,Maybe
1,0,Indian,22,1,1,Yes
2,1,Indian,31,1,0,Maybe
3,0,Indian,25,0,0,Maybe
4,1,Indian,27,0,0,Maybe
5,1,Indian,24,0,0,Yes
6,0,Indian,34,0,0,Yes
7,1,Pakistani,24,1,1,Yes
8,0,Indian,19,0,0,Yes
9,0,Indian,16,1,0,Yes


### 3.3. Codificación _one hot encoding_ <a name="one_hot_encoding"> </a>

In [113]:
# EJERCICIO: codificar mediante one hot encoding los atributos discretos con más de dos 'levels'
#    - Guardarlos en los dataframes 'ONE_HOT_NATIONALITY' y 'ONE_HOT_DESSERT' 
#    - Pandas proporciona un método para hacerlo

ONE_HOT_NATIONALITY = pd.get_dummies(DATOS[['Nationality']]).astype(int)
ONE_HOT_DESSERT = pd.get_dummies(DATOS[['Dessert']]).astype(int)
ONE_HOT_NATIONALITY[:10]

,Nationality_Algerian,Nationality_Canadian,Nationality_China,Nationality_Indian,Nationality_Indonesain,Nationality_Indonesia,Nationality_Indonesian,Nationality_Indonesian,Nationality_Japan,Nationality_Korean,...,Nationality_Maldivian,Nationality_Maldivian,Nationality_Mauritian,Nationality_Nigerian,Nationality_Pakistan,Nationality_Pakistani,Nationality_Pakistani,Nationality_Seychellois,Nationality_Tanzanian,Nationality_Yemen
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
8,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 3.4. Reducción de dimensionalidad de la codificación _one hot_ <a name="reduccion_one_hot"> </a>

Hay varias técnicas para reducir la dimensionalidad, en este notebook usaremos <code>PCA</code> (Análisis de Componentes Principales) una técnica basada en el cálculo de autovectores que busca encontrar una proyección de los datos en un nuevo espacio dimensional donde se maximice la varianza de los datos en las primeras componentes.

Determinar el número óptimo de dimensiones de salida depende en gran medida de la naturaleza de los datos. No hay una respuesta única, pero sí algunas heurísticas. Nosotros emplearemos la de la _raíz cuadrada_, que propone quedarse con un número de dimensiones igual a la raíz cuadrada de las dimensiones originales.

In [114]:
# Importación del estimador PCA
from sklearn.decomposition import PCA

In [120]:
# EJERCICIO: reducir a dos dimensiones el dataframe 'ONE_HOT_DESSERT' usando PCA

ONE_HOT_DESSERT = PCA(n_components=2).fit_transform(ONE_HOT_DESSERT)
ONE_HOT_DESSERT[:10]


array([[-0.68496279, -0.19938099],
       [ 0.72830843, -0.25099934],
       [-0.68496279, -0.19938099],
       [-0.68496279, -0.19938099],
       [-0.68496279, -0.19938099],
       [ 0.72830843, -0.25099934],
       [ 0.72830843, -0.25099934],
       [ 0.72830843, -0.25099934],
       [ 0.72830843, -0.25099934],
       [ 0.72830843, -0.25099934]])

In [127]:
# EJERCICIO: reducir a cinco dimensiones el dataframe 'ONE_HOT_NATIONALITY' usando PCA

ONE_HOT_NATIONALITY = PCA(n_components=5).fit_transform(ONE_HOT_NATIONALITY)
ONE_HOT_NATIONALITY[:10]

array([[-1.69047088e-01,  4.37369505e-03,  4.30490534e-03,
         7.09003872e-04,  1.00650922e-16],
       [-1.69047088e-01,  4.37369505e-03,  4.30490535e-03,
         7.09003872e-04, -7.30095720e-18],
       [-1.69047088e-01,  4.37369505e-03,  4.30490534e-03,
         7.09003872e-04, -1.35124360e-16],
       [-1.69047088e-01,  4.37369505e-03,  4.30490534e-03,
         7.09003872e-04, -1.35180816e-16],
       [-1.69047088e-01,  4.37369505e-03,  4.30490534e-03,
         7.09003872e-04,  2.20227367e-18],
       [-1.69047088e-01,  4.37369505e-03,  4.30490534e-03,
         7.09003872e-04, -1.94733941e-17],
       [-1.69047088e-01,  4.37369505e-03,  4.30490534e-03,
         7.09003872e-04, -1.07036769e-17],
       [ 8.19367499e-01, -1.51157526e-01, -2.92531412e-01,
        -2.05969816e-01, -3.90369984e-01],
       [-1.69047088e-01,  4.37369505e-03,  4.30490534e-03,
         7.09003872e-04, -1.51630718e-17],
       [-1.69047088e-01,  4.37369505e-03,  4.30490534e-03,
         7.09003872e-04

## 4. Cálculo de valores ausentes <a name="valores_ausentes"> </a>

Es común encontrarse con datasets en los que faltan valores en algunas columnas. Antes de despreciar la fila correspondiente, puede ser una buena estrategia rellenar esos huecos con valores razonables. Veremos algunas técnicas simples para hacer esto. 

In [69]:
# En primer lugar leeremos una versión del dataset con valores ausentes
DATOS_INCOMPLETOS = pd.read_csv('./weather_missing.csv')
print(DATOS_INCOMPLETOS)

     outlook  temperature  humidity  windy play
0      sunny           85      85.0    NaN   no
1      sunny           80      90.0    NaN   no
2   overcast           83      86.0    NaN  yes
3      rainy           70      96.0  False  yes
4      rainy           68       NaN  False  yes
5      rainy           65       NaN   True   no
6   overcast           64      65.0   True  yes
7      sunny           72      95.0  False   no
8      sunny           69      70.0  False  yes
9      rainy           75      80.0  False  yes
10     sunny           75      70.0   True  yes
11  overcast           72      90.0   True  yes
12  overcast           81      75.0  False  yes
13     rainy           71      91.0   True   no


### 4.1. Rellenar con  valores globales <a name="global"> </a>

La técnica más simple para calcular esos valores es apoyarse en las funciones media y moda. En concreto, dependiendo del tipo de atributo para el que falten valores:
- **Numérico**: media del resto de valores de la columna
- **Discreto**: valor más frecuente (moda) del resto de los valores de la columna 

In [74]:
# EJERCICIO: construir las siguientes columnas en la que se sustituyen los valores NaN de la forma en que se indica:
#    - 'humidity_fill' sustituyendo los valores NaN de 'humidity' con la media de la columna

media_humidity = DATOS_INCOMPLETOS['humidity'].mean()
DATOS_INCOMPLETOS['humidity_fill'] = DATOS_INCOMPLETOS['humidity'].fillna(media_humidity)

#    - 'windy_fill' sustituyendo los valores NaN de 'windy' con el valor más frecuente de la columna

moda_windy = moda(DATOS_INCOMPLETOS['windy'])
DATOS_INCOMPLETOS['windy_fill'] = DATOS_INCOMPLETOS['windy'].fillna(moda_windy)

DATOS_INCOMPLETOS

,outlook,temperature,humidity,windy,play,humidity_fill,windy_fill
0,sunny,85,85.0,NaN,no,85.00,False
1,sunny,80,90.0,NaN,no,90.00,False
2,overcast,83,86.0,NaN,yes,86.00,False
3,rainy,70,96.0,False,yes,96.00,False
4,rainy,68,NaN,False,yes,82.75,False
5,rainy,65,NaN,True,no,82.75,True
6,overcast,64,65.0,True,yes,65.00,True
7,sunny,72,95.0,False,no,95.00,False
8,sunny,69,70.0,False,yes,70.00,False
9,rainy,75,80.0,False,yes,80.00,False


In [76]:
# TEST: de la sustitución de valores ausentes
print(media_humidity)
print(moda_windy)
print(DATOS_INCOMPLETOS[['humidity', 'humidity_fill', 'windy', 'windy_fill']])

82.75
False
    humidity  humidity_fill  windy  windy_fill
0       85.0          85.00    NaN       False
1       90.0          90.00    NaN       False
2       86.0          86.00    NaN       False
3       96.0          96.00  False       False
4        NaN          82.75  False       False
5        NaN          82.75   True        True
6       65.0          65.00   True        True
7       95.0          95.00  False       False
8       70.0          70.00  False       False
9       80.0          80.00  False       False
10      70.0          70.00   True        True
11      90.0          90.00   True        True
12      75.0          75.00  False       False
13      91.0          91.00   True        True


### 4.2. Rellenar con valores específicos para grupos <a name="por_grupo"> </a>

La idea consiste en aprovechar la información que proporcionan otros atributos para determinar un valor específico. Por ejemplo:
- Calcular distintas medias del atributo _humidity_ en función del atributo _outlook_ (_sunny_, _overcast_, _rainy_) 
- Calcular distintas modas del atributo _windy_ en función del atributo _outlook_ (_sunny_, _overcast_, _rainy_)

El procedimiento sería:
1. Calcular grupos en función del atributo en el que nos apoyaremos (en nuestro caso _outlook_)
2. Agregar el atributo ausente (mediante moda o media, según proceda) a partir de los grupos
3. Usar los valores agregados para sustituir los valores ausentes según el atributo de apoyo

In [82]:
# EJERCICIO: sustituir los valores ausentes de 'humidity' por la media de los días con el mismo 'outlook'. Seguir estos pasos:
#    - Calcular grupos para 'datos_incompletos' en función de 'outlook' y guardarlos en 'grupos_outlook'
#    - Calcular las medias de 'humidity' para cada valor de 'outlook' y guardarlas en 'medias_humidity' 
#    - Usar la siguiente función para cambiar de forma selectiva los valores ausentes de 'humidity'
#            def rellena_humidity(fila):
#                valor = fila['humidity']
#                if np.isnan(valor):
#                    valor = medias_humidity[fila['outlook']]
#                return valor

grupos_outlook = DATOS_INCOMPLETOS.groupby('outlook')
medias_humidity = grupos_outlook['humidity'].agg('mean')

def rellena_humidity(fila):
    valor = fila['humidity']
    if np.isnan(valor):
        valor = medias_humidity[fila['outlook']]
    return valor

In [83]:
# TEST: de la sustitución selectiva de valores ausentes
print(medias_humidity)
print(DATOS_INCOMPLETOS[['humidity', 'humidity_fill']])

outlook
overcast    79.0
rainy       89.0
sunny       82.0
Name: humidity, dtype: float64
    humidity  humidity_fill
0       85.0          85.00
1       90.0          90.00
2       86.0          86.00
3       96.0          96.00
4        NaN          82.75
5        NaN          82.75
6       65.0          65.00
7       95.0          95.00
8       70.0          70.00
9       80.0          80.00
10      70.0          70.00
11      90.0          90.00
12      75.0          75.00
13      91.0          91.00
